In [1]:
#For filling out empty parts of the manifest (as indicated by lack of lane information)

In [1]:
import glob
import functools
import urllib2

import pandas as pd
import json
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import numpy as np
import shutil

import boto

In [2]:
black_list = """293XT_Input_IgG1120
293XT_Input_1120
293XT_Input_0204
293XT_CLIP_IgG_1120
293XT_CLIP_RBFOX2_1120
293XT_CLIP_RBFOX2_0204
Skin_CLIP_YB1and_DDX6
Skin_CLIP_YB1and_DDX6
Skin_CLIP_IgG
Skin_Input
Stefan_IgG_040915
Stefan_IGF2BP1_040915
Stefan_IGF2BP2_040915
Stefan_IGF2BP3_040915
Stefan_Input_040915
1_K562_Input_1_Infminus
3_K562_Input_1_Infplus
5_K562_Input_2_Infminus
7_K562_Input_2_Infplus
2_K5621_Infminus_AND_4_K5621_Infplus
2_K5621_Infminus_AND_4_K5621_Infplus
6_K5622_Infminus_AND_8_K5622_Infplus
6_K5622_Infminus_AND_8_K5622_Infplus
13_GM12878_Input_2_Infminus
15_GM12878_Input_2_Infplus
9_GM12878_Input_1_Infminus
11_GM12878_Input_1_Infplus
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus
KK_1_CLIPin_Fus
KK_2_CLIPin_Fus
KK_3_CLIPip_Fus
KK_4_CLIPip_Fus
KK_5_CLIPin_Fus
KK_6_CLIPin_Fus
KK_7_CLIPip_Fus
KK_8_CLIPip_Fus
KK_9_CLIPin_Taf
KK_10_CLIPin_Taf
KK_11_CLIPip_Taf
KK_12_CLIPip_Taf
KK_13_CLIPin_Taf
KK_14_CLIPin_Taf
KK_15_CLIPip_Taf
KK_16_CLIPip_Taf
KK_17_CLIPin_tdp
KK_18_CLIPin_tdp
KK_19_CLIPip_tdp
KK_20_CLIPip_tdp
KK_21_CLIPin_tdp
KK_22_CLIPin_tdp
KK_23_CLIPip_tdp
KK_24_mnCLIP_TDP43_IP_low_2""".split("\n")

In [3]:
json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest.is_encode = manifest.is_encode == "TRUE"
manifest.is_4000 = manifest.is_4000 == "TRUE"

manifest.method_Paper_flag = manifest.Method_Paper_flag == "TRUE" 
manifest['exp_id'] = manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])
#manifest = manifest.drop(u'', axis=1) #Drops empty columns to try and get rid of a bug



In [4]:
#Returns R1 file of any file that matches the name of the short hiseq file name in the dir
def get_matching_file(short_hiseq_name, file_location):
    files_in_dir = glob.glob(os.path.join(file_location, "*_R1_*"))
    files_in_dir = [os.path.basename(full_name) for full_name in files_in_dir]

    #don't know why I took this appraoch the first time...
    #matching_files = [full_name for full_name in files_in_dir if os.path.basename(full_name).startswith(short_hiseq_name)]

    matching_files = [full_name for full_name in files_in_dir if "_".join(full_name.split("_")[:-4]) == short_hiseq_name]
    
    if len(matching_files) > 1:
        print matching_files, short_hiseq_name
        print "double match, trying to identify the experiment with most exact match ...",
        matching_files = [item for item in files_in_dir if "_".join(os.path.basename(item).split("_")[:-4]) == "FOX2_SS4in3B_CLIP" ]    
        if len(matching_files) == 1:
            print "found"
        else:
            print "not found"
    if len(matching_files) != 1:
        print "error match wrong"
        print matching_files, short_hiseq_name
    matching_file = matching_files[0] 
    return matching_file

def get_sample_number(row):
    short_hiseq_name = row.Hiseq_file_name
    file_location = row.file_location

    matching_file = get_matching_file(short_hiseq_name, file_location)
    sample_number = os.path.basename(matching_file).split(".")[0].split("_")[-4]
    
    return "{}_{}".format(short_hiseq_name, sample_number)

def get_lane_number(row):
    short_hiseq_name = row.Hiseq_file_name
    file_location = row.file_location
    
    matching_file = get_matching_file(short_hiseq_name, file_location)
    lane_number = os.path.basename(matching_file).split(".")[0].split("_")[-3]
    
    return lane_number

In [5]:
filtered_files = manifest[(manifest.Lane == "") & ~(manifest.Hiseq_file_name.isin(black_list))].copy()

In [6]:
filtered_files["Lane"] = filtered_files.apply(get_lane_number, axis=1)
filtered_files['Hiseq_file_name'] = filtered_files.apply(get_sample_number, axis=1)

In [7]:
filtered_files.to_csv("/home/gpratt/Dropbox/EricGabe_ENCODE/tmp_manifest_update.csv")